# One-Bit GEMM With TVM!

On and off for the past few weeks I've been playing with TVM, to get a sense of its capabilities and figure out whether it's a technology we can take advantage of going forward.  It's already been used to good effect on some FB workloads (for example, text-to-speech), so it's not new to FB, but it is relatively new to me (and probably much of the Glow team).

# Why?

Glow only sort of has a kernel compiler.  We rely on a library of hand-written ops, with a few custom passes (e.g. kernel stacking) and a healthy dose of help from LLVM's generally good loop optimizer and vectorizer.  But this leaves a code generation gap: each operator must be implemented for each target architecture and each precision level by hand, creating a potentially problematic cross-product.  In addition there's the usual problem with hand-tuning kernels, in that it's pretty laborious to explore options.  I actually became interested in TVM around the time I started looking at writing custom kernels for Habana, since I thought it might be easier for me to write a code generator than to use all the new intrinsics correctly ;-).

# What?

So what have I done with TVM?  Mostly I've approached it from a user point-of-view, rather than an infra POV.  I did dig into the implementation a bit (more on that later), but I wanted to see what the kernel authoring and optimizing experience was like.  Andrew Tulloch was kind enough to give me a few project ideas, so I picked up the idea of optimizing 1-bit quantized matrix multiplication.

## Background (Really, 1 bit?)

If you have vectors of 0s and 1s you can really quickly do a dot product via `popcount(A & B)`.  Since popcount is implemented as an x86 instruction, it seems like this could be really fast.  Let's leave aside accuracy questions for now (although it can be surprisingly OK!).  I wanted to see how fast I could make a 1-bit GEMM using TVM.

# Kernel Definition

TVM follows Halide in separating the "compute" definition (what result you want to produce) from the "schedule" (essentially, the optimized loop nest).  Compute expressions are functions on the output indices.  Here's a 1-bit GEMM, which I've arbitrarily decided should be an $8 \times 8192$ by $8192 \times 8$ multiplication, to really stress those bitwise ops.  This definition states that each element of the output tensor `C` should be a reduction over the `qk` axis, which will perform `popcount(A[m, qk] & B[n, qk])`.  It took me a little getting used to the "axis" and "reduction" concepts (though that passed quickly) but it helped a lot to dump the IR in pseudocode form using `tvm.lower`, as seen in this example.

In [1]:
import tvm
import numpy as np

def bgemm_compute(M, N, K, dtype, target):
    QK = K // (8 * np.dtype(dtype).itemsize)
    A = tvm.placeholder((M, QK), dtype=dtype, name="A")
    B = tvm.placeholder((N, QK), dtype=dtype, name="B")
    qk = tvm.reduce_axis((0, QK), name="qk")
    C = tvm.compute(
        (M, N),
        lambda m, n: tvm.sum(
            tvm.popcount(A[m, qk] & B[n, qk]).astype(dtype),
            axis=[qk]))
    s = tvm.create_schedule(C.op)
    return s, [A, B, C]

def bgemm(M, N, K, dtype, target):
    s, bufs = bgemm_compute(M, N, K, dtype, target)
    fn = tvm.build(s, bufs, target=target)
    return fn, s, bufs

fn, s, bufs = bgemm(8, 8, 8192, dtype="uint32", target="llvm")
print(tvm.lower(s, bufs, simple_mode=True))


produce compute {
  for (m, 0, 8) {
    for (n, 0, 8) {
      compute[((m*8) + n)] = (uint32)0
      for (qk, 0, 256) {
        compute[((m*8) + n)] = (compute[((m*8) + n)] + popcount(bitwise_and(A[((m*256) + qk)], B[((n*256) + qk)])))
      }
    }
  }
}



## Aside: Bitpacking with TOPI

TOPI (TVM Operator Inventory) is a library of operator implementations.  There happens to already be a bitpacking routine (there's a GEMM, too, so a lot of this whole exercise can be short-circuited with TOPI, but leave that aside for now).  Let's use their bitpack implementation to create bitpacked matrices:

In [2]:
import topi.nn

def bitpack(Anp, dtype="uint32"):
    A = tvm.placeholder(Anp.shape, name="A", dtype=str(Anp.dtype))
    B = topi.nn.bitpack(A, 1, 1, 1, dtype)
    s = tvm.create_schedule(B.op)
    fn = tvm.build(s, [A, B], "llvm -mcpu=core-avx2")
    ctx = tvm.context("llvm -mcpu=core-avx2")
    Bshape = topi.util.get_const_tuple(B.shape)
    b = tvm.nd.array(np.zeros(Bshape, dtype=dtype), ctx)
    fn(tvm.nd.array(Anp, ctx), b)
    return b.asnumpy().reshape(b.shape[0], b.shape[2])

## Test correctness/performance with numpy

In [3]:
def test_bgemm(bgemm, M, N, K, dtype, target):
    a = np.random.randint(0, 2, (M, K))
    b = np.random.randint(0, 2, (N, K))
    pa = bitpack(a, dtype)
    pb = bitpack(b, dtype)
    fn, s, bufs = bgemm(M, N, K, dtype, target)
    ctx = tvm.context(target)
    
    # Check correctness
    c = tvm.nd.array(np.zeros((pa.shape[0], pb.shape[0]), dtype=dtype), ctx)
    fn(tvm.nd.array(pa, ctx), tvm.nd.array(pb, ctx), c)
    tvm.testing.assert_allclose(c.asnumpy(), a.dot(b.T))
    
    # Check performance
    evaluator = fn.time_evaluator(fn.entry_name, ctx, number=100)
    time = evaluator(tvm.nd.array(pa, ctx), tvm.nd.array(pb, ctx), c).mean
    print("{} {}/{}/{} GFLOPS: {}".format(
        bgemm.__name__,
        M, N, K,
        2 * a.shape[0] * a.shape[1] * b.shape[0] / time / 1e9))
    return fn, s, bufs
    
test_bgemm(bgemm, 8, 8, 1024, "uint32", "llvm");

bgemm 8/8/1024 GFLOPS: 32.780620538907485


That's... kinda not that great.  Doesn't my laptop have hardware popcount?  What's going on?  Let's inspect the [LLVM bitcode](https://github.com/bertmaher/bitserial/blob/master/bgemm.ll).

In [4]:
with open("bgemm.ll", "w") as f:
    f.write(fn.get_source("ll"))


OK, I see `llvm.ctpop`, so that's not the problem.  Let's check the [assembly](https://github.com/bertmaher/bitserial/blob/master/bgemm.s):

In [5]:
with open("bgemm.s", "w") as f:
    f.write(fn.get_source("asm"))

Oh.  I don't see anything from AVX2.  And why are we accumulating int32s anyways?  Let's fix those problems:

In [6]:
fn, s, bufs = test_bgemm(bgemm, 8, 8, 1024, "uint64", "llvm -mcpu=core-avx2")

bgemm 8/8/1024 GFLOPS: 317.24271468680416


Much better.  Now let's try something interesting.  TVM makes it easy to experiment with loop tiling:

In [7]:
def bgemm_tiled(M, N, K, dtype, target):
    s, bufs = list(bgemm_compute(M, N, K, dtype, target))
    C = bufs[2]
    m, n = C.op.axis
    s[C].tile(m, n, 2, 2)
    fn = tvm.build(s, bufs, target=target)
    return fn, s, bufs

fn, s, bufs = test_bgemm(bgemm_tiled, 8, 8, 1024, "uint64", "llvm -mcpu=core-avx2")

bgemm_tiled 8/8/1024 GFLOPS: 260.31140768986336


Wait, that's worse than before.  Let's try a common trick from the HPC playbook and create a *write cache*, so that the tiles of C can be computed in registers before being written back out to memory.

In [8]:
def bgemm_tiled_cached(M, N, K, dtype, target):
    s, bufs = list(bgemm_compute(M, N, K, dtype, target))
    C = bufs[2]
    CC = s.cache_write(C, "global")
    m, n = C.op.axis
    mo, no, mi, ni = s[C].tile(m, n, 2, 2)
    s[CC].compute_at(s[C], no)
    mc, nc = s[CC].op.axis
    qk, = s[CC].op.reduce_axis
    s[CC].reorder(qk, mc, nc)
    fn = tvm.build(s, bufs, target=target)
    return fn, s, bufs

fn, s, bufs = test_bgemm(bgemm_tiled_cached, 8, 8, 1024, "uint64", "llvm -mcpu=core-avx2")

bgemm_tiled_cached 8/8/1024 GFLOPS: 399.8291745470075


Now we're talking!  As you've probably noticed, there's a bit of an aspect of trial-and-error to this process.  Bitserial algorithms are especially sensitive, because LLVM has some special tricks (similar to, though not precisely this [AVX2 lookup-table based method](https://github.com/WojciechMula/sse-popcount/blob/master/popcnt-avx2-lookup.cpp)).  To quickly search this space, TVM provides some nice auto-tuning support.  You can write a "template" function that defines the config space, which TVM will search for you.  In the interest of brevity I won't list the template in this notebook, but you can check it out [in this repo](https://github.com/bertmaher/bitserial/blob/master/tune_bgemm.py).  The best schedule I've found (for these sizes) computes 2x1 tiles in cache, and clocks around 500 GFLOPS on my MacBook.

# Ludicrous Speed

What?  500 GFLOPS isn't enough for you?  Let's go to... 600.  For really long vectors, a [Harley-Seal](https://github.com/WojciechMula/sse-popcount/blob/master/popcnt-avx2-harley-seal.cpp) popcount is usually the fastest algorithm around.  We can use TVM's `tensorize` primitive to dispatch to a custom C++ implementation of HS.

## Define the tensorization

First we define the tensorized operator using a compute expression.  We bind TVM's buffers to native pointers using `tvm.decl_buffer`.

In [9]:
def intrin_popcnt_hs2(QK, dtype):
    a = tvm.placeholder((1, QK), dtype=dtype, name="hsA")
    b = tvm.placeholder((1, QK), dtype=dtype, name="hsB")
    qk = tvm.reduce_axis((0, QK), name='qk')
    c = tvm.compute(
        (1, 1),
        lambda m, n: tvm.sum(
            tvm.popcount(a[m, qk] & b[n, qk]).astype(dtype), axis=[qk]),
        name="c")
    ab = tvm.decl_buffer(a.shape, a.dtype, name="A", offset_factor=1, strides=[tvm.var("s"), 1])
    bb = tvm.decl_buffer(b.shape, b.dtype, name="B", offset_factor=1, strides=[tvm.var("s"), 1])
    cb = tvm.decl_buffer(c.shape, c.dtype, name="C", offset_factor=1, strides=[tvm.var("s"), 1])
    def intrin_func(ins, outs):
        ib = tvm.ir_builder.create()
        aa, bb = ins
        cc, = outs
        ib.emit(tvm.call_extern("uint64", "popcnt_hs2",
                                cc.access_ptr("w"),
                                aa.access_ptr("r"),
                                bb.access_ptr("r"),
                                QK // 4))
        return ib.get()
    with tvm.build_config(offset_factor=1):
        return tvm.decl_tensor_intrin(c.op, intrin_func, binds={a: ab, b: bb, c: cb})


## Define the native code

Now here's the C++ implementation of Harley-Seal.  There are nicer ways to do this than an inline string, but for ipython purposes here it is:

In [10]:
def popcnt_hs2_impl():
    cpp_code = """
#include <immintrin.h>

__m256i popcount(const __m256i v)
{
  const __m256i m1 = _mm256_set1_epi8(0x55);
  const __m256i m2 = _mm256_set1_epi8(0x33);
  const __m256i m4 = _mm256_set1_epi8(0x0F);

  const __m256i t1 = _mm256_sub_epi8(v,       (_mm256_srli_epi16(v,  1) & m1));
  const __m256i t2 = _mm256_add_epi8(t1 & m2, (_mm256_srli_epi16(t1, 2) & m2));
  const __m256i t3 = _mm256_add_epi8(t2, _mm256_srli_epi16(t2, 4)) & m4;
  return _mm256_sad_epu8(t3, _mm256_setzero_si256());
}

void CSA(__m256i& h, __m256i& l, __m256i a, __m256i b, __m256i c)
{
  const __m256i u = a ^ b;
  h = (a & b) | (u & c);
  l = u ^ c;

}

extern "C" uint64_t popcnt_hs2(uint64_t* C, const uint64_t* Ai, const uint64_t* Bi, const uint32_t size)
{
  const __m256i *A = (const __m256i *)Ai;
  const __m256i *B = (const __m256i *)Bi;
  __m256i total     = _mm256_setzero_si256();
  __m256i ones      = _mm256_setzero_si256();
  __m256i twos      = _mm256_setzero_si256();
  __m256i fours     = _mm256_setzero_si256();
  __m256i eights    = _mm256_setzero_si256();
  __m256i sixteens  = _mm256_setzero_si256();
  __m256i twosA, twosB, foursA, foursB, eightsA, eightsB;

  const uint64_t limit = size - size % 16;
  uint64_t i = 0;

  for(; i < limit; i += 16)
  {
    CSA(twosA, ones, ones, A[i+0] & B[i+0], A[i+1] & B[i+1]);
    CSA(twosB, ones, ones, A[i+2] & B[i+2], A[i+3] & B[i+3]);
    CSA(foursA, twos, twos, twosA, twosB);
    CSA(twosA, ones, ones, A[i+4] & B[i+4], A[i+5] & B[i+5]);
    CSA(twosB, ones, ones, A[i+6] & B[i+6], A[i+7] & B[i+7]);
    CSA(foursB, twos, twos, twosA, twosB);
    CSA(eightsA,fours, fours, foursA, foursB);
    CSA(twosA, ones, ones, A[i+8] & B[i+8], A[i+9] & B[i+9]);
    CSA(twosB, ones, ones, A[i+10] & B[i+10], A[i+11] & B[i+11]);
    CSA(foursA, twos, twos, twosA, twosB);
    CSA(twosA, ones, ones, A[i+12] & B[i+12], A[i+13] & B[i+13]);
    CSA(twosB, ones, ones, A[i+14] & B[i+14], A[i+15] & B[i+15]);
    CSA(foursB, twos, twos, twosA, twosB);
    CSA(eightsB, fours, fours, foursA, foursB);
    CSA(sixteens, eights, eights, eightsA, eightsB);

    total = _mm256_add_epi64(total, popcount(sixteens));
  }

  total = _mm256_slli_epi64(total, 4);     // * 16
  total = _mm256_add_epi64(total, _mm256_slli_epi64(popcount(eights), 3)); // += 8 * ...
  total = _mm256_add_epi64(total, _mm256_slli_epi64(popcount(fours),  2)); // += 4 * ...
  total = _mm256_add_epi64(total, _mm256_slli_epi64(popcount(twos),   1)); // += 2 * ...
  total = _mm256_add_epi64(total, popcount(ones));

  for(; i < size; i++)
    total = _mm256_add_epi64(total, popcount(A[i] & B[i]));


  *C = static_cast<uint64_t>(_mm256_extract_epi64(total, 0))
       + static_cast<uint64_t>(_mm256_extract_epi64(total, 1))
       + static_cast<uint64_t>(_mm256_extract_epi64(total, 2))
       + static_cast<uint64_t>(_mm256_extract_epi64(total, 3));
  return 0;
}
"""
    from tvm.contrib import util, clang
    temp = util.tempdir()
    ll_path = temp.relpath("temp.ll")
    ll_code = clang.create_llvm(cpp_code, output=ll_path, options=["-march=native"])
    return ll_code


## Tensorize!

Now we can apply our tensorization.  This one is particularly easy because the Harley-Seal popcount already applies to the innermost loop, so we don't have to reshape the compute expression at all.

In [11]:
def bgemm_hs2(M, N, K, dtype="uint64", target="llvm"):
    QK = K // (8 * np.dtype(dtype).itemsize)
    A = tvm.placeholder((M, QK), dtype=dtype, name="A")
    B = tvm.placeholder((N, QK), dtype=dtype, name="B")
    qk = tvm.reduce_axis((0, QK), name="qk")
    C = tvm.compute(
        (M, N), 
        lambda m, n: tvm.sum(
            tvm.popcount(A[m, qk] & B[n, qk]).astype(dtype),
            axis=[qk]))
    s = tvm.create_schedule(C.op)
    m, n = s[C].op.axis
    qk, = s[C].op.reduce_axis
    s[C].tensorize(qk, intrin_popcnt_hs2(QK, dtype=dtype))
    s[C].pragma(m, "import_llvm", popcnt_hs2_impl())

    fn = tvm.build(s, [A, B, C], target=target)
    return fn, s, [A, B, C]


In [12]:
test_bgemm(bgemm_hs2, 2, 2, 65536, "uint64", "llvm -mcpu=core-avx2");

bgemm_hs2 2/2/65536 GFLOPS: 605.9801893225765
